# Task 1

1. ¿Cómo afecta la elección de la estrategia de exploración (exploring starts vs soft policy) a la precisión de la evaluación de políticas en los métodos de Monte Carlo?
- Considere la posibilidad de comparar el desempeño de las políticas evaluadas con y sin explorar los inicios o con diferentes niveles de exploración en políticas blandas.
    
La elección de la estrategia de exploración afecta la precisión de la evaluación de políticas en los métodos de Monte Carlo de las siguientes maneras:

Exploring Starts: Esta estrategia asegura que cada estado inicial y acción posible tenga una probabilidad positiva de ser explorada. Esto significa que en la evaluación de políticas, se cubrirá todo el espacio de estados y acciones posibles, lo que puede llevar a una estimación más precisa del valor de la política en general. Sin embargo, esto puede ser computacionalmente costoso y puede llevar más tiempo para converger si el espacio de estados es grande.

Soft Policy (Política Blanda): Las políticas blandas, como las estrategias epsilon-greedy, permiten cierta probabilidad de exploración, lo que implica que no siempre se tomará la acción de mayor valor esperado. Esto ayuda a explorar el espacio de estados y acciones, pero con un nivel de control sobre la cantidad de exploración. Cuanto mayor sea el nivel de exploración (por ejemplo, mayor epsilon), más se explorará, lo que puede llevar a una estimación más precisa pero menos eficiente del valor de la política. Menor exploración (menor epsilon) puede llevar a una estimación más rápida pero menos precisa.


2. En el contexto del aprendizaje de Monte Carlo fuera de la póliza, ¿cómo afecta la razón de muestreo de importancia a la convergencia de la evaluación de políticas? Explore cómo la razón de muestreo de importancia afecta la estabilidad y la convergencia.

En el contexto del aprendizaje de Monte Carlo fuera de la póliza, la razón de muestreo de importancia es crucial para corregir el sesgo introducido por el hecho de que los datos no se recopilan según la política objetivo. Esta razón ajusta las estimaciones de valor para reflejar con precisión la política que se está evaluando.

Convergencia y Estabilidad: Una alta razón de muestreo de importancia puede llevar a varianzas muy altas, lo que puede causar inestabilidad en las estimaciones de valor y ralentizar la convergencia. Sin embargo, asegura que las estimaciones sean no sesgadas. Para mejorar la estabilidad, se pueden utilizar técnicas como la normalización o el recorte de las razones de importancia.

3. ¿Cómo puede el uso de una soft policy influir en la eficacia del aprendizaje de políticas óptimas en comparación con las políticas deterministas en los métodos de Monte Carlo? Compare el desempeño y los resultados de aprendizaje de las políticas derivadas de estrategias épsilon-greedy con las derivadas de
políticas deterministas.

Políticas Epsilon-Greedy: Estas políticas permiten explorar aleatoriamente acciones que no son óptimas con una probabilidad epsilon, lo que facilita la exploración de diferentes trayectorias y la obtención de mejores políticas a largo plazo. Sin embargo, pueden ser menos eficientes en términos de convergencia y podrían aprender más lentamente debido a la exploración constante de acciones subóptimas.

Políticas Deterministas: Estas políticas siempre seleccionan la acción de mayor valor esperado según el conocimiento actual, lo que puede llevar a una convergencia más rápida en escenarios donde la política óptima se encuentra rápidamente. Sin embargo, pueden quedar atrapadas en soluciones subóptimas si el espacio de estados no se explora adecuadamente.

Las políticas blandas exploran más y pueden encontrar mejores soluciones a largo plazo, aunque más lentamente. Las políticas deterministas convergen más rápido pero pueden quedar atrapadas en soluciones subóptimas.

4. ¿Cuáles son los posibles beneficios y desventajas de utilizar métodos de Monte Carlo off-policy en comparación con los on-policy en términos de eficiencia de la muestra, costo computacional. y velocidad de aprendizaje?

Eficiencia de la Muestra:

- Off-Policy: Permiten utilizar datos de políticas antiguas o diferentes, lo que puede ser útil para aprovechar datos históricos. Sin embargo, la eficiencia puede verse afectada debido a la necesidad de corregir el sesgo mediante la razón de muestreo de importancia.
- On-Policy: Utilizan datos de la política actual, lo que asegura que las muestras sean directamente relevantes para la política que se está evaluando, pero requiere generar nuevos datos para cada política, lo que puede ser menos eficiente.

Costo Computacional:

- Off-Policy: Puede ser más costoso debido a la necesidad de calcular razones de muestreo de importancia y manejar varianzas altas.
- On-Policy: Puede ser menos costoso en términos de computación, pero requiere datos frescos, lo que puede aumentar los costos de recolección de datos.

Velocidad de Aprendizaje:

- Off-Policy: Puede ser más lento debido a la alta varianza en las estimaciones, pero permite utilizar más datos potenciales.
- On-Policy: Generalmente más rápido y directo, pero puede requerir más muestras para cubrir adecuadamente el espacio de estados.

# Task 2

In [1]:
# Importar Librerias
import numpy as np
import random
from collections import defaultdict

In [2]:
# Definición de la clase InventoryEnvironment
class InventoryEnvironment:
    def __init__(self):
        self.products = ['product_A', 'product_B']
        self.max_stock = 10
        self.demand = {'product_A': [0, 1, 2], 'product_B': [0, 1, 2]}
        self.restock_cost = {'product_A': 5, 'product_B': 7}
        self.sell_price = {'product_A': 10, 'product_B': 15}
        self.state = None

    def reset(self):
        self.state = {product: random.randint(0, self.max_stock) for product in self.products}
        return self.state

    def step(self, action):
        reward = 0
        for product in self.products:
            stock = self.state[product]
            restock = action[product]
            self.state[product] = min(self.max_stock, stock + restock)
            demand = random.choice(self.demand[product])
            sales = min(demand, self.state[product])
            self.state[product] -= sales
            reward += sales * self.sell_price[product] - restock * self.restock_cost[product]
        return self.state, reward

# Inicialización del entorno
env = InventoryEnvironment()


In [3]:
def generate_episode(policy, env, max_days=30):
    state = env.reset()
    episode = []
    for _ in range(max_days):
        action = policy(state)
        next_state, reward = env.step(action)
        episode.append((state, action, reward, next_state))
        state = next_state
    return episode

def random_policy(state):
    return {product: random.randint(0, env.max_stock - state[product]) for product in env.products}


In [4]:
def exploring_starts(env, num_episodes):
    episodes = []
    for _ in range(num_episodes):
        initial_state = env.reset()
        initial_action = random_policy(initial_state)
        env.state = initial_state
        episode = generate_episode(lambda _: initial_action, env)
        episodes.append(episode)
    return episodes


In [5]:
def epsilon_greedy_policy(state, Q, actions, epsilon=0.1):
    if random.random() < epsilon:
        # Exploración: seleccionar una acción aleatoria
        return random.choice(actions)
    else:
        # Explotación: seleccionar la mejor acción basada en los valores Q
        max_q = float('-inf')
        best_action = None
        for action in actions:
            q_value = Q.get((str(state), str(action)), 0)  # Obtener el valor Q para (estado, acción)
            if q_value > max_q:
                max_q = q_value
                best_action = action
        return best_action


In [6]:
def off_policy_mc_control(env, num_episodes, gamma=0.9, epsilon=0.1):
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    Q = defaultdict(float)

    # Definir las posibles acciones
    actions = [{'product_A': a, 'product_B': b} for a in range(env.max_stock + 1) for b in range(env.max_stock + 1)]

    def policy(state):
        return epsilon_greedy_policy(state, Q, actions, epsilon)

    for _ in range(num_episodes):
        episode = generate_episode(policy, env)
        G = 0
        for t in reversed(range(len(episode))):
            state, action, reward, _ = episode[t]
            G = gamma * G + reward
            if (str(state), str(action)) not in [(str(x[0]), str(x[1])) for x in episode[:t]]:
                returns_sum[(str(state), str(action))] += G
                returns_count[(str(state), str(action))] += 1
                Q[(str(state), str(action))] = returns_sum[(str(state), str(action))] / returns_count[(str(state), str(action))]
    return Q


In [7]:
# Ejecución de exploring starts
episodes = exploring_starts(env, num_episodes=100)

# Evaluación con aprendizaje off-policy
Q = off_policy_mc_control(env, num_episodes=500, epsilon=0.1)

# Visualización de algunos valores Q estimados
print("Algunos valores Q estimados:")
for key in list(Q.keys())[:5]:
    print(f"Estado-Acción: {key}, Valor Q: {Q[key]}")


Algunos valores Q estimados:
Estado-Acción: ("{'product_A': 0, 'product_B': 0}", "{'product_A': 7, 'product_B': 1}"), Valor Q: 32.222
Estado-Acción: ("{'product_A': 0, 'product_B': 0}", "{'product_A': 4, 'product_B': 6}"), Valor Q: 51.898855027211994
Estado-Acción: ("{'product_A': 0, 'product_B': 0}", "{'product_A': 0, 'product_B': 0}"), Valor Q: 99.57764118423802
Estado-Acción: ("{'product_A': 2, 'product_B': 6}", "{'product_A': 4, 'product_B': 3}"), Valor Q: 24.970000000000013
Estado-Acción: ("{'product_A': 2, 'product_B': 6}", "{'product_A': 0, 'product_B': 7}"), Valor Q: -17.296869999999984


# Respuestas a las Preguntas del Laboratorio

## Pregunta 1: ¿Cuál es el valor estimado de mantener diferentes niveles de existencias para cada producto?

Los valores Q estimados proporcionan una expectativa de ganancias para diferentes combinaciones de niveles de inventario y acciones de reposición. Por ejemplo, para el estado `{'product_A': 0, 'product_B': 0}`, el valor Q con la acción `{'product_A': 7, 'product_B': 1}` es aproximadamente 32.22. Esto indica que, con un stock inicial de 0 unidades tanto de `product_A` como de `product_B` y reponiendo 7 unidades de `product_A` y 1 unidad de `product_B`, se espera una ganancia promedio de 32.22.

Otra combinación, como la acción `{'product_A': 4, 'product_B': 6}` en el mismo estado inicial, tiene un valor Q de aproximadamente 51.90, lo que sugiere que reponer esta cantidad de productos es más rentable. Sin embargo, la acción de no reponer (`{'product_A': 0, 'product_B': 0}`) presenta el valor Q más alto de 99.58, indicando que, en este escenario, no reponer es la estrategia más rentable, posiblemente debido a la evitación de costos de reposición. Por otro lado, para el estado `{'product_A': 2, 'product_B': 6}`, reponer hasta `{'product_A': 0, 'product_B': 7}` resulta en un valor Q negativo de -17.30, lo que sugiere pérdidas debido a un exceso de inventario o costos que superan las ganancias esperadas.


## Pregunta 2: ¿Cómo afecta el valor epsilon en la política blanda al rendimiento?

El valor epsilon en la política epsilon-greedy controla el balance entre exploración y explotación. Un epsilon mayor fomenta la exploración, lo que permite al algoritmo probar más acciones aleatorias y potencialmente descubrir estrategias más efectivas. Sin embargo, un epsilon demasiado alto puede reducir las recompensas inmediatas al tomar acciones subóptimas con mayor frecuencia.

En nuestro codigo, utilizamos un epsilon de 0.1, lo que permitió una exploración moderada. Si el valor epsilon fuera demasiado bajo, el modelo podría quedarse atascado en políticas subóptimas. En cambio, si fuera demasiado alto, se podrían tomar demasiadas acciones subóptimas, disminuyendo las recompensas obtenidas.

## Pregunta 3: ¿Cuál es el impacto de utilizar el aprendizaje fuera de la política en comparación con el aprendizaje dentro de la política?

El aprendizaje off-policy permite evaluar una política objetivo utilizando datos generados por una política de comportamiento diferente. Esto es particularmente útil cuando no es factible o seguro seguir directamente la política objetivo. Off-policy puede ser más flexible y aprovechar datos históricos o generados por simulaciones.

En comparación, el aprendizaje on-policy requiere que el agente siga la política que se está evaluando, lo que puede limitar la capacidad de explorar diferentes estrategias. El uso de una política epsilon-greedy en nuestro caso permitió al algoritmo explorar una gama más amplia de acciones, facilitando la identificación de políticas más rentables.
